In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa, ec
from cryptography.hazmat.primitives.asymmetric.ec import ECDSA
from cryptography.hazmat.primitives.asymmetric.utils import Prehashed
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.exceptions import InvalidSignature

class DigitalSignature:
    def __init__(self, algorithm='RSA'):
        self.algorithm = algorithm.upper()

        if self.algorithm == 'RSA':
            self.private_key = rsa.generate_private_key(
                public_exponent=65537, key_size=2048
            )
            self.public_key = self.private_key.public_key()
        elif self.algorithm == 'ECDSA':
            self.private_key = ec.generate_private_key(ec.SECP256R1())
            self.public_key = self.private_key.public_key()
        elif self.algorithm == 'ECC':
            self.private_key = ec.generate_private_key(ec.SECP256R1())
            self.public_key = self.private_key.public_key()
        else:
            raise ValueError("Unsupported algorithm. Supported algorithms: RSA, ECDSA, ECC")

    def sign_message(self, message: bytes) -> bytes:
        if self.algorithm == 'RSA':
            signature = self.private_key.sign(
                message,
                padding.PSS(
                    mgf=padding.MGF1(hashes.SHA256()),
                    salt_length=padding.PSS.MAX_LENGTH
                ),
                hashes.SHA256()
            )
        elif self.algorithm == 'ECDSA':
            signature = self.private_key.sign(
                message, ECDSA(hashes.SHA256())
            )
        else:
            raise ValueError("Unsupported algorithm for signing.")
        return signature

    def verify_signature(self, message: bytes, signature: bytes) -> bool:
        try:
            if self.algorithm == 'RSA':
                self.public_key.verify(
                    signature,
                    message,
                    padding.PSS(
                        mgf=padding.MGF1(hashes.SHA256()),
                        salt_length=padding.PSS.MAX_LENGTH
                    ),
                    hashes.SHA256()
                )
            elif self.algorithm == 'ECDSA':
                self.public_key.verify(
                    signature, message, ECDSA(hashes.SHA256())
                )
            else:
                raise ValueError("Unsupported algorithm for verification.")
            return True
        except InvalidSignature:
            return False

    def perform_key_exchange(self, peer_public_key):

        if self.algorithm != 'ECC':
            raise ValueError("Key exchange is only supported for ECC.")

        # Perform ECDH key exchange
        shared_key = self.private_key.exchange(ec.ECDH(), peer_public_key)
        return shared_key

if __name__ == "__main__":
    print("Testing RSA...")
    rsa_ds = DigitalSignature(algorithm='RSA')
    rsa_message = b"Payment of $1000 to Anvesha."
    rsa_signature = rsa_ds.sign_message(rsa_message)
    print(f"Message: {rsa_message.decode()}")
    print("RSA Signature created successfully.")
    if rsa_ds.verify_signature(rsa_message, rsa_signature):
        print(f"RSA Signature verified: The message '{rsa_message.decode()}' is authentic.")
    else:
        print("RSA Signature verification failed!")

    print("\nTesting ECDSA...")
    ecdsa_ds = DigitalSignature(algorithm='ECDSA')
    ecdsa_message = b"Payment of $1000 to Aryana."
    ecdsa_signature = ecdsa_ds.sign_message(ecdsa_message)
    print(f"Message: {ecdsa_message.decode()}")
    print("ECDSA Signature created successfully.")
    if ecdsa_ds.verify_signature(ecdsa_message, ecdsa_signature):
        print(f"ECDSA Signature verified: The message '{ecdsa_message.decode()}' is authentic.")
    else:
        print("ECDSA Signature verification failed!")

    print("\nTesting ECC (Key Exchange)...")
    ecc_ds1 = DigitalSignature(algorithm='ECC')
    ecc_ds2 = DigitalSignature(algorithm='ECC')

    shared_key1 = ecc_ds1.perform_key_exchange(ecc_ds2.public_key)
    shared_key2 = ecc_ds2.perform_key_exchange(ecc_ds1.public_key)

    print("Shared key computed by both parties matches:", shared_key1 == shared_key2)

    tampered_message = b"Unauthorized payment of $5000 to A."
    print("\nTampering test for RSA...")
    if rsa_ds.verify_signature(tampered_message, rsa_signature):
        print(f"RSA Signature verified: The tampered message '{tampered_message.decode()}' is authentic (this should not happen).")
    else:
        print("RSA Tampering detected: The message integrity is compromised!")

    print("\nTampering test for ECDSA...")
    if ecdsa_ds.verify_signature(tampered_message, ecdsa_signature):
        print(f"ECDSA Signature verified: The tampered message '{tampered_message.decode()}' is authentic (this should not happen).")
    else:
        print("ECDSA Tampering detected: The message integrity is compromised!")

Testing RSA...
Message: Payment of $1000 to Anvesha.
RSA Signature created successfully.
RSA Signature verified: The message 'Payment of $1000 to Anvesha.' is authentic.

Testing ECDSA...
Message: Payment of $1000 to Aryana.
ECDSA Signature created successfully.
ECDSA Signature verified: The message 'Payment of $1000 to Aryana.' is authentic.

Testing ECC (Key Exchange)...
Shared key computed by both parties matches: True

Tampering test for RSA...
RSA Tampering detected: The message integrity is compromised!

Tampering test for ECDSA...
ECDSA Tampering detected: The message integrity is compromised!


Block 0:
  Hash: 000059653997cad50958b749584f55f473dd99301f35e6b94ea8bb29e03d08b2
  Prev: 0
  Transactions: Genesis Block
  Nonce: 3125
  Reward: 50
  Difficulty: 4
--------------------------------------------------
Block 1:
  Hash: 0000e9a3ed840fa1c7cdfb396e87441d7db1dece06d3f89de29abb3f59d95762
  Prev: 000059653997cad50958b749584f55f473dd99301f35e6b94ea8bb29e03d08b2
  Transactions: ['Aryana -> Anvesha: 5 BTC', 'Kinjal -> Gaurang: 3 BTC']
  Nonce: 37220
  Reward: 50
  Difficulty: 4
--------------------------------------------------
Block 2:
  Hash: 00000dda86768487b717f921599b2b1b8dbd83bf05ba7875d0602d8466c0e17e
  Prev: 0000e9a3ed840fa1c7cdfb396e87441d7db1dece06d3f89de29abb3f59d95762
  Transactions: ['Aryana -> Anvesha: 5 BTC', 'Kinjal -> Gaurang: 3 BTC']
  Nonce: 275406
  Reward: 50
  Difficulty: 5
--------------------------------------------------
Block 3:
  Hash: 000000cb4d51fd57caaf0fbbc8c214cfa2d4101edd4fc987627fa5507ccdea0d
  Prev: 00000dda86768487b717f921599b2b1b8dbd83bf05ba787